Tài liệu này mang giấy phép Creative Commons Attribution (CC BY). (c) Nguyễn Ngọc Sáng, Zhukovsky 06/2019.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

# Bài 20. Cấu trúc dữ liệu Cells, Sides

Có rất nhiều dữ liệu cần phải lưu trữ, sắp xếp để tiện việc sử dụng. Ở bài này ta sẽ tạo lớp dữ liệu `các ô lưới - Cells` và `các bề mặt - Sides` bằng phương thức `Class` trong Python. 

## 1. Lớp dữ liệu `Các ô lưới - Cells`

### a. Lớp dữ liệu `Ô lưới - Cell`
Trước hết ta nhớ lại công thức xác định biến bảo toàn từ bài 18:
$$U^{n+1} = U^n + \frac{\Delta t}{V_{ABCD}}\sum {\vec F.\vec S_n} \qquad (1)$$
<img src='img\Bai_20_1.png' width = 300>
Để thuận tiện trong việc lưu trữ, sử dụng dữ liệu ta sẽ tạo một lớp **Ô lưới - Cell** chứa các thông tin (thuộc tính): `tọa độ tâm, thể tích, biến nguyên thủy, biến bảo toàn, bước thời gian, tổng các dòng đi qua các bề mặt`. Thông số hình học của ô lưới được xác định bởi `4 đỉnh lưới`: ô lưới thứ `(j,i)` gồm 4 đỉnh `[(j,i), (j,i+1), (j+1,i+1), (j+1,i)]`. Khi đó ta có công thức xác định tâm và thể tích ô lưới:

$$(x,y)_{center} = \frac{(x,y)_1+(x,y)_2+(x,y)_3+(x,y)_4}{4}$$
$$V_{cell} = \frac{|[(x,y)_1-(x,y)_3] \times [(x,y)_2-(x,y)_4]|}{2}$$

Ngoài ra, để xác định bước thời gian trong từng ô lưới ta cần kích thước ô lưới. Bước thời gian trong từng ô lưới được xác định theo quy tắc để đảm bảo `sóng xuất hiện trên biên từ bài toán phân rã gián đoạn chưa đủ thời gian để chạy tới biên đối diện` (xem lại ở phần 2) tức là: $dt = CFL \times min(\frac{dx}{\lambda^x_{max}}, \frac{dy}{\lambda^y_{max}}), CFL<1$

Việc xác định bước thời gian trong từng ô lưới khá phức tạp do còn phụ thuộc vào hình dạng ô lưới và sóng tạo ra trên các biên. Để đơn giản ta sử dụng điều kiện sau và tùy chỉnh bước thời gian theo CFL nếu xảy ra không hội tụ:
$dt = \frac{min(dx, dy)}{|u|+a}$ với dx, dy là khoảng cách tâm hai mặt đối diện, |u| - độ lớn vận tốc dòng chảy, a - vận tốc âm thanh. Ta định nghĩa kích thước ô lưới chính bằng $min(dx, dy)$.

**Module data.py:**

In [1]:
# coding: utf-8
# module data.py 
import numpy as np
from functions import U2P # Hàm tính U từ P trong module functions.py 

# tâm ô lưới bằng trung bình cộng tọa độ bốn đỉnh lưới
def center(vertices):
    return sum(vertices) / 4.

# diện tích ô lưới bằng một nửa độ lớn tích có hướng hai vector đường chéo
# ~ tương đương thể tích ô lưới trong trường hợp 3 chiều
def volume(vertices):
    return abs(np.cross(vertices[0] - vertices[2], vertices[1] - vertices[3]) / 2.)

# kích thước ô lưới
def cell_size(vertices):
    dx_vec = (vertices[1]-vertices[0] + vertices[2] - vertices[3])/2.
    dy_vec = (vertices[2]-vertices[1] + vertices[3] - vertices[0])/2.
    dx = dx_vec.dot(dx_vec)**0.5
    dy = dy_vec.dot(dy_vec)**0.5
    return min(dx, dy)

# lớp dữ liệu ô lưới:
# tại thời điểm khởi tạo, giá trị P, U chưa xác định (None)
class Cell:
    # Hàm khởi tạo, khai báo
    def __init__(self, vertices): #4 đỉnh lưới theo thứ tự ngược chiều KĐH
        self.center   = center(vertices)
        self.volume   = volume(vertices)
        self.size     = cell_size(vertices)
        self.P = None           #(rho, u, v, p)
        self.U = None           #(rho, rho*u, rho*v, rho*e); e= p/(rho*km1) + (u**2 + v**2)/2
        self.res = np.zeros(4)  #tổng các dòng đi qua các bề mặt: sum(F*S)
        self.dt  = 0.0          #bước thời gian

In [2]:
#ví dụ: xác định ô lưới với bốn đỉnh lần lượt có tọa độ [0., 0.], [1., 0.], [1., 1.], [0., 1.]
vertices = np.array([[0., 0.], [1., 0.], [1., 1.], [0., 1.]])
c = Cell(vertices)
print(c.center, c.volume)

(array([ 0.5,  0.5]), 1.0)


Như vậy, chúng ta đã có lớp dữ liệu ô lưới với những thông số cơ bản nhất, trong trường hợp chương trình được mở rộng hay giải một hệ phương trình khác, các thông số trong ô lưới có thể thay đổi,thêm bớt.

### b. Lớp dữ liệu `Các ô lưới - Cells`

 Để giải một bài toán cần rất nhiều ô lưới, do đó ta cần xây dựng thêm lớp dữ liệu **Các ô lưới - Cells**. Lớp `Các ô lưới` bao gồm một `dãy các ô lưới` cùng với một số hàm cơ bản: `lấy phần tử của dãy ô lưới; xác định bước thời gian cục bộ, toàn cục; thực hiện bước lặp xác định U (theo công thức 1) trong từng ô lưới; xác định giá trị P mới`.

In [12]:
# Để xác định các ô lưới ta cần kích thước lưới NjxNi và tọa độ các điểm lưới points
class Cells():
    # Khởi tạo
    def __init__(self, Nj, Ni, points):
        self.size  = [Nj, Ni] # Kích thước lưới
        self.len   = Nj*Ni    # Tổng số ô lưới
        self.cells = []       # Dãy 1D chứa các ô lưới
        for j in range(Nj):
            for i in range(Ni):
                #ô lưới thứ (j,i) gồm 4 đỉnh [(j,i), (j,i+1), (j+1,i+1), (j+1,i)] (hình 1)
                vers = (points[j, i], points[j, i + 1], points[j + 1, i + 1], points[j + 1, i])
                # khởi tạo ô lưới và thêm vào dãy các ô lưới
                self.cells.append(Cell(vers))

    # Ba phương thức cơ bản để lấy các phần tử của dãy các ô lưới:
    def __getitem__(self, item):
        # Lấy một đoạn các ô lưới: cells[start:stop]
        if isinstance(item, slice):
            return self.cells.__getslice__(item.start, item.stop)
        
        # Lấy ô lưới thứ (j,i): cells[j,i]
        elif isinstance(item, tuple):
            j, i = item
            if(j < 0): j += self.size[0]
            if(i < 0): i += self.size[1]
            return self.cells[j*self.size[1]+i]
        
        # Lấy ô lưới thứ j*i: cells[j*i]
        elif isinstance(item, int):
            return self.cells[item]
    
    # tính bước thời gian cục bộ trong từng ô lưới 
    def time_step_cell(selfs):
        for cell in selfs.cells:
            a = (1.4 * cell.P[3] / cell.P[0]) ** 0.5  # vận tốc âm thanh
            v = (cell.P[1] ** 2 + cell.P[2] ** 2) ** 0.5  # vận tốc dòng chảy
            cell.dt = cell.size/(v + a)

    # Xác định bước thời gian cho toàn vùng tính toán (toàn cục)
    def time_step_global(self, CFL):
        # trước hết cần xác định bước thời gian trong từng ô lưới
        self.time_step_cell()
        # sau đó tìm bước thời gian nhỏ nhất trong các ô lưới
        dt = 1e6
        # tìm bước thời gian nhỏ nhất trong các ô lưới
        for cell in self.cells:
            dt = min(dt, cell.dt)          
        return CFL*dt
    
    # Thực hiện bước lặp: xác định U ở bước thời gian tiếp theo
    def new_U(self, dt):
        for cell in self.cells:
            # print('cell.ress ', cell.res)
            cell.U += dt/cell.volume*cell.res # công thức (1)
            cell.res[:] = 0.0                 # sau khi xác định U, đưa giá trị res về 0.0

    # Xác định P từ U
    def new_P(self):
        for cell in self.cells:
            U2P(cell.U, cell.P) # Hàm U2P

In [1]:
# Trong module functions.py:
# Hàm U2P: xác định biến biên thủy P từ biến bảo toàn U
def U2P(U, P):
    P[0] = U[0]
    P[1] = U[1] / U[0]
    P[2] = U[2] / U[0]
    P[3] = (U[3] - 0.5 * P[0] * (P[1] ** 2 + P[2] ** 2)) * (1.4 - 1) #gamma = 1.4

**Bài tập**: dùng hàm `import_mesh` để xác định Nj, Ni, points như ở bài 19; khởi tạo một dãy Cells và kiểm tra.

# 2. Lớp dữ liệu `Bề mặt - Side`

<img src='img\Bai_20_2.png'>

**Chú ý quan trọng**
<br>Quy ước: 
- `Mỗi mặt có hai ô lưới hai bên trái phải (theo chiều vector pháp tuyến: từ trái sang phải)`.
- `khi dòng đi qua bề mặt thì tổng dòng ở ô lưới bên trái bị mất đi (-),ở ô lưới bên phải được thêm vào (+):` $$cell_{left}.res \ -= \ Flux,\  cell_{right}.res \ += \ Flux$$

Quy trước trên giúp ta không bị nhầm lẫn, sai sót khi tính tổng các dòng đi qua các bề mặt của ô lưới. Hãy so sánh tổng dòng `res` trong ô lưới với hai trường hợp sau:
<img src='img\Bai_20_3.png'>

Ta thấy hai trường hợp là như nhau. Do đó để đơn giản trong việc xác định các thông số bề mặt, ta sẽ sử dụng trường hợp 2 (**vector pháp tuyến cùng chiều với hai trục x, y**) để xây dựng lớp dữ liệu bề mặt.

Lớp dữ liệu Bề mặt chứa các thuộc tính cơ bản: `diện tích, vector pháp tuyến đơn vị, con trỏ tới hai ô lưới hai bên`.
Trong trường hợp hai chiều, bề mặt được quy định bởi hai điểm $(x,y)_1, (x,y)_2$, hay vector bề mặt: $\vec S = (x,y)_2 - (x,y)_1 = (dx, dy)$. Khi đó:
<br>diện tích bề mặt (2D- độ dài): $S = |\vec S|$
<br>vector pháp tuyến đơn vị:      $\vec n = (-dy, dx)/S$

In [14]:
#sử dụng tích vô hướng hai vector .dot để tính chiều dài vector
#~ tương đương diện tích bề mặt trong trường hợp 3 chiều
def area(side_vec):
    return (side_vec.dot(side_vec))**0.5

#xác định vector pháp tuyến của bề mặt: S*n, với n - vector pháp tuyến đơn vị
def normal(side_vec):
    return np.array([side_vec[1], -side_vec[0]])

In [16]:
#định nghĩa lớp bề mặt
class Side:
    def __init__(self, side_vec):
        self.area   = area(side_vec) # diện tích bề mặt
        self.normal = normal(side_vec)/self.area   #vector pháp tuyến đơn vị
        self.cells = None   #hai ô lưới hai bên trái phải, tạm thời chưa xác định
        # Quy ước: self.cells[0] - ô bên trái, self.cells[1] - ô bên phải

Chia các bề mặt làm hai loại: **bề mặt ở trong vùng tính toán** (có hai ô lưới hai bên) và **bề mặt trên các biên** (chỉ có một ô lưới kề bên).Ta cần thêm lớp dữ liệu để chứa tất cả các bề mặt. Ta có thể tạo lớp Sides như sau:

In [17]:
'''
Quy ước:
                 boundary_3
                  <--------    
              ^             ^
    boundary_0| inner_sides |boundary_1
                  <--------
                 boundary_2  
'''

class Sides:   
    # để khởi tạo Sides cần tọa độ điểm lưới về địa chỉ các ô lưới
    def __init__(self, points, cells):
        #xác định các vector bề mặt từ các điểm lưới 
        sides_i = points[:, :-1] - points[:, 1:]   # sides nằm ngang
        sides_j = points[1:] - points[:-1]         # sides thẳng đứng

        # Biên_0 gồm các mặt ở cột đầu sides_j
        # Ô lưới bên trái không xác định, bên phải là các ô ở cột đầu tiên 
        self.boundary_0 = []
        for j in range(sides_j.shape[0]):
            side = Side(sides_j[j, 0])
            side.cells = [None, cells[j, 0]]
            self.boundary_0.append(side)

        # Biên_1 gồm các mặt ở cột cuối sides_j
        # Ô lưới bên phải không xác định, bên trái là các ô ở cột cuối
        self.boundary_1 = []
        for j in range(sides_j.shape[0]):
            side = Side(sides_j[j, -1])
            side.cells = [cells[j, -1], None]
            self.boundary_1.append(side)
            
        # Biên_2 gồm các mặt ở hàng đầu sides_i
        # Ô lưới bên phải không xác định, bên trái là các ô ở hàng đầu
        self.boundary_2 = []
        for i in range(sides_i.shape[1]):
            side = Side(sides_i[0, i])
            side.cells = [None, cells[0, i]]
            self.boundary_2.append(side)

        # Biên_3 gồm các mặt ở hàng cuối sides_i
        # Ô lưới bên trái không xác định, bên phải là các ô ở hàng cuối
        self.boundary_3 = []
        for i in range(sides_i.shape[1]):
            side = Side(sides_i[-1, i])
            side.cells = [cells[-1, i], None]
            self.boundary_3.append(side)

        # Những hàng còn lại bên trong sides_i
        self.inner_sides = []
        for j in range(1, sides_i.shape[0] - 1):
            for i in range(sides_i.shape[1]):
                side = Side(sides_i[j, i])
                side.cells = [cells[j - 1, i], cells[j, i]]
                self.inner_sides.append(side)

        # Những cột còn lại bên trong sides_j
        for i in range(1, sides_j.shape[1] - 1):
            for j in range(sides_j.shape[0]):
                side = Side(sides_j[j, i])
                side.cells = [cells[j, i - 1], cells[j, i]]
                self.inner_sides.append(side)
    
    # Hàm đặt điều kiện biên 
    def set_boco(self, boco):
        self.boco = boco # boco là tập hợp các điều kiện biên

    # Hàm xác định dòng qua biên
    def flux_boundary(self):
        # Tương ứng 4 biên, có 4 điều kiện biên (trường hợp đơn giản)
        # Các chỉ số 1, 0, 1, 0 là chỉ số của ô lưới kề biên side.cells[id]
        self.boco[0](self.boundary_0, 1) # ô lưới kề biên ở bên phải
        self.boco[1](self.boundary_1, 0) # ô lưới kề biên ở bên trái
        self.boco[2](self.boundary_2, 1) # ô lưới kề biên ở bên phải
        self.boco[3](self.boundary_3, 0) # ô lưới kề biên ở bên trái

    # Hàm xác định dòng qua các biên bên trong
    def flux_inner(self, flux_func): #flux_func: hàm tính dòng qua từng mặt (side)
        for side in self.inner:
            F = flux_func(side, side.cells[0].P, side.cells[1].P)
            # Như đã quy ước ở trên:
            side.cells[0].res -= F # ô bên trái -
            side.cells[1].res += F # ô bên phải +

Liên quan đến điều kiện biên và hàm tính flux sẽ được ở những bài tiếp theo. Bài 20 kết thúc tại đây, hãy lưu các hàm, lớp vào file `data.py` và suy nghĩ về những ý tưởng, quy ước, cách xây dựng các lớp dữ liệu.

# [Bài 21. Điều kiện biên `supersonic, wall`](Bai_21.ipynb)